# Holdings from multiple systems

In [129]:
import os
import math
import json
import pytz

from datetime import datetime, timedelta
from typing import List, Tuple, Dict
from collections import namedtuple

import lusid
import pandas as pd

from lusid import models as models
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from IPython.core.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [130]:
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transactions_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

## 1. Create portfolios

In [131]:
from lusid.exceptions import ApiException

def check_portfolio(scope, code):
    try:
        portfolios_api.delete_portfolio(scope=scope, code=code)
        return 'Portfolio successfully deleted'
    except ApiException as e:
        return 'Portfolio does not yet exist'

In [132]:
from datetime import date
import lusid.models as lm

def create_portfolio(portfolio_code, scope='', display_name='', base_currency='GBP'):
    check_portfolio(scope=scope, code=portfolio_code)
    request = lm.CreateTransactionPortfolioRequest(
        display_name=display_name,
        code=portfolio_code,
        base_currency=base_currency,
        created=str(date(2021, 10, 10)))
    
    try:
        result = transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=request)
        return result
    except ApiException as e:
        display(json.loads(e.body)['title'])

In [133]:
port_1250_response = create_portfolio('1350', scope='Bleugrain', display_name='Portfolio 1350', base_currency='GBP')
port_1232_response = create_portfolio('1332', scope='Bleugrain', display_name='Portfolio 1332', base_currency='GBP')
port_6350_response = create_portfolio('6450', scope='Bleugrain', display_name='Portfolio 6450', base_currency='GBP')

## 3. Create instruments

In [134]:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

In [135]:
InstrumentSpec = namedtuple('InstrumentSpec', ['Figi', 'Name'])

instruments = [
    InstrumentSpec("BBG000FD8G46", "HISCOX LTD"),
    InstrumentSpec("BBG000DW76R4", "ITV PLC"),
    InstrumentSpec("BBG000PQKVN8", "MONDI PLC"),
    InstrumentSpec("BBG000BDWPY0", "NEXT PLC"),
    InstrumentSpec("BBG000BF46Y8", "TESCO PLC"),
]

instruments_to_create = {
    i.Figi: models.InstrumentDefinition(
        name=i.Name, identifiers={ 'Figi': models.InstrumentIdValue(value=i.Figi) }
    ) for i in instruments
}

response = instruments_api.upsert_instruments(request_body=instruments_to_create)

instrument_ids = sorted([i.lusid_instrument_id for i in response.values.values()])
instrument_ids

['LUID_00003D6R',
 'LUID_00003D6U',
 'LUID_00003D6V',
 'LUID_00003D6W',
 'LUID_00003D6X']

## 4. Create transactions

In [136]:
import random

transactions = [
    models.TransactionRequest(
        transaction_id=f'TransactionId_{_id}',
        type='Buy',
        instrument_identifiers={ 'Instrument/default/LusidInstrumentId': _id },
        transaction_date=datetime.now(pytz.UTC).isoformat(),
        settlement_date=(datetime.now(pytz.UTC) + timedelta(days=2)).isoformat(),
        units=math.floor(random.random() * 100),
        total_consideration=lusid.CurrencyAndAmount(
            math.floor(random.random() * 1000), 'GBP')
    )
    for _id in instrument_ids
]

In [137]:
scope = 'Bleugrain'
portfolio_code = '1350'

# response = transactions_api.upsert_transactions(
#    scope, portfolio_code, transactions
# )

## 5. Add property definitions

In [138]:
def create_property(code, domain='Transaction', scope='default', required=False):
    try:
        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time='Perpetual',
                value_required=required,
                data_type_id=models.resource_id.ResourceId(
                    scope='system',
                    code='string',
                )
            )
        )
    
    except ApiException as e:
        detail = json.loads(e.body)
        print(detail)
        if detail['code'] != 124:
            raise e

In [139]:
property_domain = 'Holding'

common_scope = 'common'
ps_scope = 'ps'
crims_scope = 'crims'

default_properties = ['A', 'B']
ps_properties = ['C', 'D']
crims_properties = ['E', 'F']

In [140]:
# create default properties common to PS and CRIMS
create_property(default_properties[0], domain=property_domain, scope=common_scope, required=True)
create_property(default_properties[1], domain=property_domain, scope=common_scope, required=True)

create_property(ps_properties[0], domain=property_domain, scope=ps_scope)
create_property(ps_properties[1], domain=property_domain, scope=ps_scope)

create_property(crims_properties[0], domain=property_domain, scope=crims_scope)
create_property(crims_properties[1], domain=property_domain, scope=crims_scope)

{'name': 'PropertyAlreadyExists', 'errorDetails': [], 'code': 124, 'type': 'https://docs.lusid.com/#section/Error-Codes/124', 'title': "Error creating Property Definition 'Holding/common/A' because it already exists.", 'status': 400, 'detail': "Error creating Property Definition 'Holding/common/A' because it already exists.", 'instance': 'https://bleugrain.lusid.com/app/insights/logs/0HMEB1IH10EUD:00000007', 'extensions': {}}
{'name': 'PropertyAlreadyExists', 'errorDetails': [], 'code': 124, 'type': 'https://docs.lusid.com/#section/Error-Codes/124', 'title': "Error creating Property Definition 'Holding/common/B' because it already exists.", 'status': 400, 'detail': "Error creating Property Definition 'Holding/common/B' because it already exists.", 'instance': 'https://bleugrain.lusid.com/app/insights/logs/0HMEB1IR0J1DD:00000004', 'extensions': {}}
{'name': 'PropertyAlreadyExists', 'errorDetails': [], 'code': 124, 'type': 'https://docs.lusid.com/#section/Error-Codes/124', 'title': "Erro

## Set/Adjust holdings

In [141]:
def set_holdings(portfolio_code, instrument_identifier, units, amount, properties, scope='Bleugrain'):
    current_datetime = datetime.now(pytz.UTC)
    print(f'Current Time: {current_datetime}')
    
    adjust_holding_request = models.AdjustHoldingRequest(
        instrument_identifiers={
            'Instrument/default/Figi': instrument_identifier
        },
        properties=properties,
        tax_lots=[
            models.TargetTaxLotRequest(
                units=units,
                cost=models.CurrencyAndAmount(amount=amount, currency='GBP'),
                portfolio_cost=amount,
                price=math.floor(amount/units),
                purchase_date=current_datetime.isoformat(),
                settlement_date=(current_datetime + timedelta(days=2)).isoformat(),
            )
        ],
        currency='GBP'
    )
    transactions_api.set_holdings(scope, 
                                  portfolio_code, 
                                  effective_at=datetime.now(pytz.UTC).isoformat(), 
                                  adjust_holding_request=[adjust_holding_request])

In [142]:
properties = {
    'Holding/common/A': models.PerpetualProperty('Holding/common/A', 
                                                 models.PropertyValue(label_value='A_Value')),
    'Holding/common/B': models.PerpetualProperty('Holding/common/B', 
                                                 models.PropertyValue(label_value='B_Value')),
    'Holding/ps/C': models.PerpetualProperty('Holding/ps/C', 
                                             models.PropertyValue(label_value='C_Value')),
    'Holding/ps/D': models.PerpetualProperty('Holding/ps/D', 
                                             models.PropertyValue(label_value='D_Value')),
}

set_holdings(portfolio_code, 'BBG000FD8G46', 24000, 960000, properties)

Current Time: 2021-12-30 05:00:50.150474+00:00


In [143]:
holdings_response = transactions_api.get_holdings(scope, portfolio_code)
lusid_response_to_data_frame(
    holdings_response,
    rename_properties=True
)

,instrument_scope,instrument_uid,sub_holding_keys,A(common-Properties),B(common-Properties),C(ps-Properties),D(ps-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
0,default,LUID_00003D6V,{},A_Value,B_Value,C_Value,D_Value,1350,Bleugrain,P,"24,000.00","24,000.00","960,000.00",GBP,"960,000.00",GBP,GBP


In [144]:
def adjust_holdings(portfolio_code, instrument_identifier, units, amount, properties, scope='Bleugrain'):
    current_datetime = datetime.now(pytz.UTC)
    print(f'Current Time: {current_datetime}')
    adjust_holding_request = models.AdjustHoldingRequest(
        instrument_identifiers={
            'Instrument/default/Figi': instrument_identifier
        },
        properties=properties,
        tax_lots=[
            models.TargetTaxLotRequest(
                units=units,
                cost=models.CurrencyAndAmount(amount=amount, currency='GBP'),
                portfolio_cost=amount,
                price=math.floor(amount/units),
                purchase_date=current_datetime.isoformat(),
                settlement_date=(current_datetime + timedelta(days=2)).isoformat(),
            )
        ],
        currency='GBP'
    )
    
    try:
        transactions_api.adjust_holdings(scope, 
                                  portfolio_code, 
                                  effective_at=datetime.now(pytz.UTC).isoformat(), 
                                  adjust_holding_request=[adjust_holding_request])
    except ApiException as e:
        raise e

In [145]:
properties = {
    'Holding/common/A': models.PerpetualProperty('Holding/common/A', 
                                                 models.PropertyValue(label_value='A_Value')),
    'Holding/common/B': models.PerpetualProperty('Holding/common/B', 
                                                 models.PropertyValue(label_value='B_Value')),
    'Holding/ps/C': models.PerpetualProperty('Holding/ps/C', 
                                             models.PropertyValue(label_value='C_Value')),
    'Holding/ps/D': models.PerpetualProperty('Holding/ps/D', 
                                             models.PropertyValue(label_value='D_Value')),
}

adjust_holdings(portfolio_code, 'BBG000DW76R4', 30000, 1800000, properties)

Current Time: 2021-12-30 05:01:32.584643+00:00


In [146]:
transactions_api.get_holdings(scope, portfolio_code)

{'href': 'https://bleugrain.lusid.com/api/api/transactionportfolios/Bleugrain/1350/holdings?effectiveAt=2021-12-30T05%3A01%3A54.8733943%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://bleugrain.lusid.com/api/api/portfolios/Bleugrain/1350',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://bleugrain.lusid.com/api/api/schemas/entities/PortfolioHolding',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://bleugrain.lusid.com/api/api/schemas/properties?propertyKeys=Holding%2Fcommon%2FA%2CHolding%2Fcommon%2FB%2CHolding%2Fps%2FC%2CHolding%2Fps%2FD%2CHolding%2Fdefault%2FSourcePortfolioId%2CHolding%2Fdefault%2FSourcePortfolioScope',
            'method': 'GET',
            'relation': 'PropertySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs rel

In [147]:
"""
properties = {
    'Holding/common/A': models.PerpetualProperty('Holding/common/A', 
                                                 models.PropertyValue(label_value='A_Value')),
    'Holding/common/B': models.PerpetualProperty('Holding/common/B', 
                                                 models.PropertyValue(label_value='B_Value')),
    'Holding/ps/C': models.PerpetualProperty('Holding/ps/C', 
                                             models.PropertyValue(label_value='C1_Value')),
    'Holding/crims/E': models.PerpetualProperty('Holding/crims/E', 
                                             models.PropertyValue(label_value='E_Value')),
    'Holding/crims/F': models.PerpetualProperty('Holding/crims/F', 
                                             models.PropertyValue(label_value='F_Value')),
}

adjust_holdings(portfolio_code, 'BBG000DW76R4', 50000, 3500000, properties)
"""

"\nproperties = {\n    'Holding/common/A': models.PerpetualProperty('Holding/common/A', \n                                                 models.PropertyValue(label_value='A_Value')),\n    'Holding/common/B': models.PerpetualProperty('Holding/common/B', \n                                                 models.PropertyValue(label_value='B_Value')),\n    'Holding/ps/C': models.PerpetualProperty('Holding/ps/C', \n                                             models.PropertyValue(label_value='C1_Value')),\n    'Holding/crims/E': models.PerpetualProperty('Holding/crims/E', \n                                             models.PropertyValue(label_value='E_Value')),\n    'Holding/crims/F': models.PerpetualProperty('Holding/crims/F', \n                                             models.PropertyValue(label_value='F_Value')),\n}\n\nadjust_holdings(portfolio_code, 'BBG000DW76R4', 50000, 3500000, properties)\n"

In [148]:
holdings_response = transactions_api.get_holdings(scope, portfolio_code)
lusid_response_to_data_frame(
    holdings_response,
    rename_properties=True
)

,instrument_scope,instrument_uid,sub_holding_keys,A(common-Properties),B(common-Properties),C(ps-Properties),D(ps-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
0,default,LUID_00003D6V,{},A_Value,B_Value,C_Value,D_Value,1350,Bleugrain,P,"24,000.00","24,000.00","960,000.00",GBP,"960,000.00",GBP,GBP
1,default,LUID_00003D6U,{},A_Value,B_Value,C_Value,D_Value,1350,Bleugrain,P,"30,000.00","30,000.00","1,800,000.00",GBP,"1,800,000.00",GBP,GBP


## Merge properties and adjust holdings

In [166]:
holdings = transactions_api.get_holdings(scope, portfolio_code)
holdings_properties = { holding.instrument_uid: holding.properties for holding in holdings.values }
holdings_properties

{'LUID_00003D6V': {'Holding/common/A': {'effective_from': datetime.datetime(2021, 12, 30, 5, 0, 50, 150814, tzinfo=tzutc()),
   'effective_until': None,
   'key': 'Holding/common/A',
   'value': {'label_value': 'A_Value',
             'label_value_set': None,
             'metric_value': None}},
  'Holding/common/B': {'effective_from': datetime.datetime(2021, 12, 30, 5, 0, 50, 150814, tzinfo=tzutc()),
   'effective_until': None,
   'key': 'Holding/common/B',
   'value': {'label_value': 'B_Value',
             'label_value_set': None,
             'metric_value': None}},
  'Holding/ps/C': {'effective_from': datetime.datetime(2021, 12, 30, 5, 0, 50, 150814, tzinfo=tzutc()),
   'effective_until': None,
   'key': 'Holding/ps/C',
   'value': {'label_value': 'C_Value',
             'label_value_set': None,
             'metric_value': None}},
  'Holding/ps/D': {'effective_from': datetime.datetime(2021, 12, 30, 5, 0, 50, 150814, tzinfo=tzutc()),
   'effective_until': None,
   'key': 'Holding/

In [152]:
try:
    type(holdings.values[0].properties['Holding/common/G'])

except KeyError as e:
    print('Property not found')

Property not found


In [153]:
instrument_identifier = 'BBG000DW76R4'
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
instrument = instruments_api.get_instrument(identifier_type='Figi', identifier=instrument_identifier)
lusid_instrument_id = instrument.lusid_instrument_id
lusid_instrument_id

'LUID_00003D6U'

In [154]:
existing_properties = holdings_properties[lusid_instrument_id]
existing_properties

{'Holding/common/A': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/A',
  'value': {'label_value': 'A_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/common/B': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/B',
  'value': {'label_value': 'B_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/C': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/C',
  'value': {'label_value': 'C_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/D': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/D',
  'value': {'label_value': 'D_V

In [155]:
all_properties = [
    'Holding/common/A',
    'Holding/common/B',
    'Holding/ps/C',
    'Holding/ps/D',
    'Holding/crims/E',
    'Holding/crims/F',
]

In [156]:
new_properties = {
    'Holding/crims/E': models.PerpetualProperty('Holding/crims/E', 
                                             models.PropertyValue(label_value='E2_Value')),
    'Holding/crims/F': models.PerpetualProperty('Holding/crims/F', 
                                             models.PropertyValue(label_value='F2_Value')),
}

In [157]:
def merge_properties(existing_properties, new_properties, list_of_all_properties):
    properties = {}
    for property_name in list_of_all_properties:
        if property_name not in new_properties.keys():
            try: 
                properties[property_name] = existing_properties[property_name]
            except KeyError as e:
                pass
        else:
            try:
                properties[property_name] = new_properties[property_name]
            except KeyError as e:
                pass
    return properties

In [163]:
merged_properties = merge_properties(existing_properties, new_properties, all_properties)
merged_properties

{'Holding/common/A': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/A',
  'value': {'label_value': 'A_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/common/B': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/B',
  'value': {'label_value': 'B_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/C': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/C',
  'value': {'label_value': 'C_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/D': {'effective_from': datetime.datetime(2021, 12, 30, 5, 1, 32, 585340, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/D',
  'value': {'label_value': 'D_V

In [164]:
adjust_holdings(portfolio_code, 'BBG000DW76R4', 58000, 4700000, merged_properties)

Current Time: 2021-12-30 05:06:53.759425+00:00


In [165]:
holdings = transactions_api.get_holdings(scope, portfolio_code)
holdings_properties = { holding.instrument_uid: holding.properties for holding in holdings.values }
holdings_properties[instrument.lusid_instrument_id]

{'Holding/common/A': {'effective_from': datetime.datetime(2021, 12, 30, 5, 6, 53, 759716, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/A',
  'value': {'label_value': 'A_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/common/B': {'effective_from': datetime.datetime(2021, 12, 30, 5, 6, 53, 759716, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/common/B',
  'value': {'label_value': 'B_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/C': {'effective_from': datetime.datetime(2021, 12, 30, 5, 6, 53, 759716, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/C',
  'value': {'label_value': 'C_Value',
            'label_value_set': None,
            'metric_value': None}},
 'Holding/ps/D': {'effective_from': datetime.datetime(2021, 12, 30, 5, 6, 53, 759716, tzinfo=tzutc()),
  'effective_until': None,
  'key': 'Holding/ps/D',
  'value': {'label_value': 'D_V

In [ ]:
holdings_properties[instrument.lusid_instrument_id]